# 🧠 Final Project: Autoregressive Language Model
This notebook demonstrates how to build a small autoregressive language model using PyTorch and Hugging Face tools.

In [1]:
# ✅ Install dependencies
!pip install torch transformers datasets wandb

   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 1.3/204.2 MB 7.4 MB/s eta 0:00:28
    --------------------------------------- 2.9/204.2 MB 7.3 MB/s eta 0:00:28
    --------------------------------------- 4.5/204.2 MB 7.3 MB/s eta 0:00:28
   - -------------------------------------- 5.8/204.2 MB 7.2 MB/s eta 0:00:28
   - -------------------------------------- 7.3/204.2 MB 7.1 MB/s eta 0:00:28
   - -------------------------------------- 8.9/204.2 MB 7.0 MB/s eta 0:00:28
   -- ------------------------------------- 10.2/204.2 MB 7.0 MB/s eta 0:00:28
   -- ------------------------------------- 11.8/204.2 MB 7.0 MB/s eta 0:00:28
   -- ------------------------------------- 13.1/204.2 MB 7.0 MB/s eta 0:00:28
   -- ------------------------------------- 14.2/204.2 MB 6.8 MB/s eta 0:00:28
   --- ------------------------------------ 15.5/204.2 MB 6.8 MB/s eta 0:00:28
   --- ------------------------------------ 16.8/204.2 MB 6.6 MB/s

In [2]:
# 📌 Import Libraries
import torch
import torch.nn as nn
import pandas as pd
from transformers import GPT2Tokenizer
import wandb

c:\Users\Victoria\anaconda4\envs\my_env_new\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 📚 Tokenizer Setup
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

c:\Users\Victoria\anaconda4\envs\my_env_new\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Victoria\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [4]:
# 🧾 Load Dataset
with open("shakespeare.txt", "r", encoding='utf-8') as f:
    text = f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'shakespeare.txt'

In [ ]:
# 📂 Custom Dataset
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, text, tokenizer, block_size=128):
        tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        self.input_ids = tokens["input_ids"]

    def __len__(self):
        return self.input_ids.size(1) - 1

    def __getitem__(self, idx):
        return self.input_ids[0, idx:idx+128], self.input_ids[0, idx+1:idx+129]

dataset = TextDataset(text, tokenizer)

In [ ]:
# 🔧 Model Definition
class SimpleDecoderLM(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=4, num_layers=3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers)
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids).permute(1, 0, 2)
        tgt_mask = nn.Transformer().generate_square_subsequent_mask(embedded.size(0)).to(embedded.device)
        decoded = self.transformer_decoder(embedded, embedded, tgt_mask=tgt_mask)
        return self.fc_out(decoded.permute(1, 0, 2))

In [ ]:
# ⚙️ Training Loop
from torch.utils.data import DataLoader
model = SimpleDecoderLM(len(tokenizer)).to("cpu")
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

wandb.init(project="lm-final-project")

epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    loader = DataLoader(dataset, batch_size=1)
    for x, y in loader:
        logits = model(x)
        loss = loss_fn(logits.view(-1, logits.size(-1)), y.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    wandb.log({"epoch": epoch + 1, "train_loss": total_loss / len(loader)})

In [ ]:
# 💾 Save the model
torch.save(model.state_dict(), "simple_decoder_lm.pt")

In [ ]:
# 🧪 Simple Text Generation
def generate(prompt, max_new_tokens=50):
    model.eval()
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]
    for _ in range(max_new_tokens):
        logits = model(input_ids)
        next_token = torch.argmax(logits[:, -1, :], dim=-1).unsqueeze(0)
        input_ids = torch.cat([input_ids, next_token], dim=1)
    return tokenizer.decode(input_ids[0])

generate("To be or not to be")

In [5]:
import ipywidgets as widgets
widgets.IntSlider()


IntSlider(value=0)